# MiniCPM-2B 参数高效微调（LoRA）消费级单卡示例

本 notebook 是一个使用 `AdvertiseGen` 数据集对 MiniCPM-2B 进行 LoRA 微调，使其具备专业的广告生成能力的代码示例。

## 硬件需求
- 显存：24GB
- 显卡架构：安培架构（推荐）
- 内存：16GB

## 1. 准备数据集

下载 AdvertiseGen 数据集
- [Google Drive](https://drive.google.com/file/d/13_vf0xRTQsyneRKdD1bZIr93vBGOczrk/view?usp=sharing)
- [Tsinghua Cloud](https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1)

下载后的数据集格式为 `.tar.gz` 的压缩格式，接下来的操作中，假设该压缩包被置于 `finetune/data/`。


In [ ]:
# 校验文件完整性
!md5sum data/AdvertiseGen.tar.gz 

In [ ]:
# 解压数据集
!tar xvf data/AdvertiseGen.tar.gz 

In [ ]:
# 转换为 ChatML 格式
import os
import shutil
import json

input_dir = "data/AdvertiseGen"
output_dir = "data/AdvertiseGenChatML"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

for fn in ["train.json", "dev.json"]:
    data_out_list = []
    with open(os.path.join(input_dir, fn), "r") as f, open(os.path.join(output_dir, fn), "w") as fo:
        for line in f:
            if len(line.strip()) > 0:
                data = json.loads(line)
                data_out = {
                    "messages": [
                        {
                            "role": "user",
                            "content": data["content"],
                        },
                        {
                            "role": "assistant",
                            "content": data["summary"],
                        },
                    ]
                }
                data_out_list.append(data_out)
        json.dump(data_out_list, fo, ensure_ascii=False, indent=4)


## 2. 使用 LoRA 进行微调

命令行一键运行

In [ ]:
!bash lora_finetune.sh